# Imports

In [ ]:
import albumentations as a
import efficientnet_pytorch
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import torch
import torch.nn as nn

from albumentations.pytorch import ToTensor
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim import SGD 
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm_notebook as tqdm

### Versions of libs

In [ ]:
np.__version__

In [ ]:
torch.__version__

# Load data

In [ ]:
data_folder = './traffic-signs-data/'

with open(os.path.join(data_folder,'train.p'), mode='rb') as f:
    train = pickle.load(f)
with open(os.path.join(data_folder,'valid.p'), mode='rb') as f:
    valid = pickle.load(f)
with open(os.path.join(data_folder,'test.p'), mode='rb') as f:
    test = pickle.load(f)

# Data exploration

Check the stored data

In [ ]:
train.keys(), valid.keys(), test.keys()

Assign varaibles

In [ ]:
X_train, y_train, coords_train, sizes_train = train['features'], train['labels'], train['coords'], train['sizes']
X_valid, y_valid, coords_valid, sizes_valid = valid['features'], valid['labels'], valid['coords'], valid['sizes']
X_test, y_test, coords_test, sizes_test = test['features'], test['labels'], test['coords'], test['sizes']

Get basic information about the dataset

In [ ]:
print("Number of training examples =", X_train.shape[0])
print("Number of validation examples =", X_valid.shape[0])
print("Number of testing examples =", X_test.shape[0])

height, width, channels = X_train.shape[1:]
print(f"Image data shape = {width}x{height}, channels={channels}, dtype={X_train.dtype}")
n_classes = len(set(train['labels']))
print("Number of classes =", n_classes)

Check sample images

In [ ]:
def preview_image(img, text):
    plt.figure(figsize=(1,1))
    plt.title(text)
    plt.axis('off')
    plt.imshow(img)

def preview_image_from_dataset(dataset, index):
    preview_image(dataset[idx], f"Index: {index}")

for idx in [123,234,456,678]:
    preview_image_from_dataset(X_train, idx)

Check histogram of classes

In [ ]:
plt.figure(figsize=(12,4))
plt.title("Histogram of classes in dataset")

plt.hist(y_train,bins = n_classes, alpha=0.2, label = 'train')
plt.hist(y_test,bins = n_classes, alpha=0.2, label='test')
plt.hist(y_valid,bins = n_classes, alpha=0.2, label='valid')

plt.legend()
pass

Similarity of distribution of number of items per class in train/test/valid seems to be good

# Preprocessing

In [ ]:
train_augmentation = a.Compose([
    a.Normalize(
        mean=(0.5, 0.5, 0.5),
        std=(0.5, 0.5, 0.5)),
    ToTensor()
])

test_augmentation = a.Compose([
    a.Normalize(
        mean=(0.5, 0.5, 0.5),
        std=(0.5, 0.5, 0.5)),
    ToTensor()
])

In [ ]:
class TrafficSignsDataset(Dataset):
    def __init__(self, features, labels, augmentation):
        self.features = features.copy()
        self.labels  = labels.copy()
        self.augmentation = augmentation
        
    def __len__(self):
        return len(self.labels)
        
    def __getitem__(self, index, noaug=False):
        img = self.features[index]
        if noaug:
            img = img
        else:
            img = self.augmentation(image=img)['image']
        
        label = int(self.labels[index])
        
        return img, label

TODO: add comment on preprocessing

In [ ]:
train_dataset = TrafficSignsDataset(X_train, y_train, train_augmentation)
test_dataset = TrafficSignsDataset(X_test, y_test, test_augmentation)
valid_dataset = TrafficSignsDataset(X_valid, y_valid, test_augmentation)

Check if datasets work:

In [ ]:
img, label = train_dataset.__getitem__(123, noaug=True)
preview_image(img, f"Label: {label}")

# Model

Instead of experimenting with own model using one that has performance of ResNet-50 while staying extremely small:   
Model name: **EfficientNet, version "B0"**

Link to paper: https://arxiv.org/pdf/1905.11946.pdf  
Using implementation from: https://github.com/lukemelas/EfficientNet-PyTorch

In [ ]:
model = efficientnet_pytorch.EfficientNet.from_name('efficientnet-b0', override_params={'num_classes': n_classes})

Checking if sample outut has proper shape

In [ ]:
dummy = torch.zeros((1,3,32,32))
model.eval()
model(dummy).shape

# Training

### Train code

In [ ]:
class MetricsAggregator:
    def __init__(self):
        self.epoch_index = -1
        self.loss_history = {}
        self.accuracy_history = {}
    
    def epoch_start(self):
        self.epoch_index += 1
        
    def __add_loss(self, loss):
        if self.epoch_index not in self.loss_history:
            self.loss_history[self.epoch_index] = []
        self.loss_history[self.epoch_index].append(loss)
        
    def add(self, loss, pred:np.ndarray, gt:np.ndarray):
        self.__add_loss(loss)
    
    def get_mean_loss(self, samples=None):
        if samples is None:
            return np.mean(self.loss_history[self.epoch_index])
        return np.mean(self.loss_history[self.epoch_index][-samples:])

In [ ]:
def train(net, loader, aggregator, criterion, device):
    aggregator.epoch_start()
    net.train()
    
    with tqdm(desc="Train epoch progress", total=len(loader)) as p:
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            aggregator.add(float(loss.item()), outputs.cpu().detach().numpy(), labels.cpu().detach().numpy())
            batch_loss = aggregator.get_mean_loss(inputs.shape[0])
            total_loss = aggregator.get_mean_loss()
            p.desc = f"Loss:{batch_loss:0.2f} ({total_loss:0.2f})"
            p.update(1)

In [ ]:
def test(net, loader, aggregator, criterion, device):
    net.eval()
    aggregator.epoch_start()
    with torch.no_grad():
        with tqdm(desc="Eval progress", total=len(loader)) as p:
            for inputs, labels in loader:
                inputs, labels = inputs.to(device), labels.to(device)
                
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                
                aggregator.add(float(loss.item()), outputs.cpu().detach().numpy(), labels.cpu().detach().numpy())
                batch_loss = aggregator.get_mean_loss(inputs.shape[0])
                total_loss = aggregator.get_mean_loss()
                p.desc = f"Loss:{batch_loss:0.2f} ({total_loss:0.2f})"
                p.update(1)        

### Training execution

In [ ]:
train_dataloader = DataLoader(train_dataset,batch_size=4, shuffle=True)
post_train_dataloader = DataLoader(train_dataset,batch_size=4, shuffle=False)
test_dataloader = DataLoader(test_dataset,batch_size=4, shuffle=False)
valid_dataloader = DataLoader(valid_dataset,batch_size=4, shuffle=False)

In [ ]:
running_train_metrics = MetricsAggregator()
train_metrics = MetricsAggregator()
val_metrics = MetricsAggregator()

In [ ]:
device = "cpu"
model = model.to(device)

In [ ]:
epochs = 1
criterion = nn.CrossEntropyLoss()
optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=4e-5)
scheduler = ReduceLROnPlateau(optimizer, patience=3, verbose=True, min_lr=1e-8, factor=0.2)

In [ ]:
with tqdm(desc='Training', total=epochs) as p:
    for epoch in range(epochs):
        
        train(model, train_dataloader, running_train_metrics, criterion, device)
        test(model, train_dataloader, train_metrics, criterion, device)
        test(model, valid_dataloader, val_metrics, criterion, device)
        
        p.update(1)

In [ ]:
# TODO

# Visualization

### Training/validation loss curves

In [ ]:
# TODO

### External sample traffic signs

In [ ]:
# TODO